In [1]:
import pandas as pd

In [2]:
X = pd.read_csv('data/train_features.csv', index_col=0)
Y = pd.read_csv('data/train_targets.csv', index_col=0)

In [16]:
rad_cols = []
dire_cols = []
neutral_cols = []
for c in X.columns:
    if c.startswith('r'):
        rad_cols.append(c)
    elif c.startswith('d'):
        dire_cols.append(c)
    else:
        neutral_cols.append(c)

In [17]:
neutral_data = X[neutral_cols]

In [12]:
radiant_team = X[rad_cols]

In [13]:
radiant_team.columns = ['d' + c[1:] for c in radiant_team.columns]
radiant_team.head(1)

,d1_hero_id,d1_kills,d1_deaths,d1_assists,d1_denies,d1_gold,d1_lh,d1_xp,d1_health,d1_max_health,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,11,0,0,0,0,543,7,533,358,600,...,0.0,0,0,0,0,0.0,0,0,0,0


In [14]:
dire_team = X[dire_cols]
dire_team.columns = ['r' + c[1:] for c in dire_team.columns]
dire_team.head(1)

,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,...,r5_stuns,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_obs_placed,r5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,12,0,0,1,13,982,12,780,650,720,...,0.0,0,0,0,0,0.0,0,0,0,0


In [51]:
target = Y.radiant_win
inv_target = ~ target
inv_target.index = inv_target.index.map(lambda x: 'inv_' + x)
target_comb = pd.concat([target, inv_target])

In [53]:
target_comb = pd.DataFrame(target_comb)

In [41]:
# neutral_data.index = neutral_data.index.map(lambda x: 'inv_')

In [45]:
X_inv = pd.concat([neutral_data, dire_team, radiant_team], axis=1)
X_inv.index = X_inv.index.map(lambda x: 'inv_' + x)

In [47]:
X_comb = pd.concat([X, X_inv])

In [52]:
X_comb.to_csv('features/train_team_inv.csv')

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [55]:
target_comb.to_csv('features/train_team_inv_target.csv')

In [1]:
import gc

In [2]:
def team_permute(X, Y):
    rad_cols = []
    dire_cols = []
    neutral_cols = []
    for c in X.columns:
        if c.startswith('r'):
            rad_cols.append(c)
        elif c.startswith('d'):
            dire_cols.append(c)
        else:
            neutral_cols.append(c)
    neutral_data = X[neutral_cols]
    
    radiant_team = X[rad_cols]
    radiant_team.columns = ['d' + c[1:] for c in radiant_team.columns]
    
    dire_team = X[dire_cols]
    dire_team.columns = ['r' + c[1:] for c in dire_team.columns]
    
    target = Y.radiant_win
    inv_target = ~ target
    inv_target.index = inv_target.index.map(lambda x: 'inv_' + x)
    target_comb = pd.concat([target, inv_target])
    gc.collect()    
    X_inv = pd.concat([neutral_data, dire_team, radiant_team], axis=1)
    X_inv.index = X_inv.index.map(lambda x: 'inv_' + x)
    X_comb = pd.concat([X, X_inv])
    gc.collect()
    return X_comb, target_comb